In [9]:
import sqlite3
import pandas as pd
from faker import Faker
import numpy as np

# Number of samples
n = 1000

# Create SQLite database
conn = sqlite3.connect('TravelBookingDB.db')
cursor = conn.cursor()

# Drop the existing tables if they exist
cursor.execute('DROP TABLE IF EXISTS Customers;')
cursor.execute('DROP TABLE IF EXISTS Flights;')
cursor.execute('DROP TABLE IF EXISTS Bookings;')
cursor.execute('DROP TABLE IF EXISTS Payments;')
cursor.execute('DROP TABLE IF EXISTS MergedTable;')
# Create DataFrame for Customers
fake = Faker()
customer_data = pd.DataFrame({
    'Customer_ID': range(1, n+1),
    'First_Name': [fake.first_name() for _ in range(n)],
    'Last_Name': [fake.last_name() for _ in range(n)],
    'Email': [fake.email() for _ in range(n)],
    'Phone': [fake.phone_number() for _ in range(n)]
})

# Create DataFrame for Flights
flight_data = pd.DataFrame({
    'Flight_ID': range(1, n+1),
    'Airline': [fake.company() for _ in range(n)],
    'Departure_City': [fake.city() for _ in range(n)],
    'Destination_City': [fake.city() for _ in range(n)],
    'Departure_Date': [fake.date_this_year() for _ in range(n)],
    'Price': np.random.randint(100, 1000, n)
})

# Create DataFrame for Bookings
booking_data = pd.DataFrame({
    'Booking_ID': range(1, n+1),
    'Customer_ID': np.random.choice(range(1, n+1), n, replace=True),
    'Flight_ID': np.random.choice(range(1, n+1), n, replace=True),
    'Booking_Date': [fake.date_this_year() for _ in range(n)]
})

# Create DataFrame for Payments
payment_data = pd.DataFrame({
    'Payment_ID': range(1, n+1),
    'Booking_ID': np.random.choice(range(1, n+1), n, replace=True),
    'Amount': np.random.uniform(50, 500, n),
    'Payment_Date': [fake.date_this_year() for _ in range(n)]
})

# Create Customers table
cursor.execute('''
    CREATE TABLE Customers (
        Customer_ID INTEGER PRIMARY KEY,
        First_Name TEXT,
        Last_Name TEXT,
        Email TEXT,
        Phone TEXT
    );
''')
customer_data.to_sql('Customers', conn, index=False, if_exists='replace')

# Create Flights table
cursor.execute('''
    CREATE TABLE Flights (
        Flight_ID INTEGER PRIMARY KEY,
        Airline TEXT,
        Departure_City TEXT,
        Destination_City TEXT,
        Departure_Date TEXT,
        Price REAL
    );
''')
flight_data.to_sql('Flights', conn, index=False, if_exists='replace')

# Create Bookings table
cursor.execute('''
    CREATE TABLE Bookings (
        Booking_ID INTEGER PRIMARY KEY,
        Customer_ID INTEGER,
        Flight_ID INTEGER,
        Booking_Date TEXT,
        FOREIGN KEY (Customer_ID) REFERENCES Customers(Customer_ID),
        FOREIGN KEY (Flight_ID) REFERENCES Flights(Flight_ID)
    );
''')
booking_data.to_sql('Bookings', conn, index=False, if_exists='replace')

# Create Payments table
cursor.execute('''
    CREATE TABLE Payments (
        Payment_ID INTEGER PRIMARY KEY,
        Booking_ID INTEGER,
        Amount REAL,
        Payment_Date TEXT,
        FOREIGN KEY (Booking_ID) REFERENCES Bookings(Booking_ID)
    );
''')
payment_data.to_sql('Payments', conn, index=False, if_exists='replace')

# Create MergedTable
cursor.execute('''
    CREATE TABLE MergedTable AS
    SELECT 
        Customers.*,
        Flights.*,
        Bookings.*,
        Payments.*
    FROM 
        Customers
    LEFT JOIN 
        Bookings ON Customers.Customer_ID = Bookings.Customer_ID
    LEFT JOIN 
        Flights ON Bookings.Flight_ID = Flights.Flight_ID
    LEFT JOIN 
        Payments ON Bookings.Booking_ID = Payments.Booking_ID;
''')

# Commit changes
conn.commit()

# Example Queries

# Query 1: Get all bookings for a specific customer
query_example_1 = "SELECT * FROM MergedTable WHERE Customer_ID = 1;"

# Query 2: Get total payments for each customer
query_example_2 = '''
    SELECT 
        Customer_ID,
        First_Name,
        Last_Name,
        SUM(Amount) AS Total_Payments
    FROM 
        MergedTable
    GROUP BY 
        Customer_ID;
'''
# Tables Creation (Previous Code)
# ...

# Example Query 3: Retrieve booking details with payment information
query_example_3 = '''
    SELECT Bookings.*, Payments.*
    FROM Bookings
    LEFT JOIN Payments ON Bookings.Booking_ID = Payments.Booking_ID;
'''

# Ethical Considerations Implementation (e.g., encryption, access controls)
# ...

# Example Query 4: Retrieve customer information with masked sensitive data
query_example_4 = '''
    SELECT Customer_ID, First_Name, Last_Name, '*****' AS Masked_Email, '*****' AS Masked_Phone
    FROM Customers;
'''


# Execute example queries
result_example_1 = pd.read_sql_query(query_example_1, conn)
result_example_2 = pd.read_sql_query(query_example_2, conn)
result_example_3 = pd.read_sql_query(query_example_3, conn)
result_example_4 = pd.read_sql_query(query_example_4, conn)

# Display example query results
print("\nExample Query 1 Result:")
print(result_example_1)

print("\nExample Query 2 Result:")
print(result_example_2)
print("\nExample Query 3 Result:")
print(result_example_3)
print("\nExample Query 4 Result:")
print(result_example_4)
# Close the connection
conn.close()


Example Query 1 Result:
   Customer_ID First_Name Last_Name                   Email  \
0            1     Samuel   Schmidt  moodynorma@example.net   

               Phone Flight_ID Airline Departure_City Destination_City  \
0  651-769-7123x7709      None    None           None             None   

  Departure_Date Price Booking_ID Customer_ID:1 Flight_ID:1 Booking_Date  \
0           None  None       None          None        None         None   

  Payment_ID Booking_ID:1 Amount Payment_Date  
0       None         None   None         None  

Example Query 2 Result:
     Customer_ID First_Name Last_Name  Total_Payments
0              1     Samuel   Schmidt             NaN
1              2    Ricardo   Johnson             NaN
2              3      Kelly    Sparks      126.798406
3              4      Jacob   Ramirez             NaN
4              5     Edward      Diaz             NaN
..           ...        ...       ...             ...
995          996       John    Franco      344.

In [8]:
import pandas as pd
from faker import Faker
import numpy as np

# Number of samples
n = 1000

# Create a Faker instance for generating fake data
fake = Faker()
# Drop the existing tables if they exist
cursor.execute('DROP TABLE IF EXISTS Customers;')
cursor.execute('DROP TABLE IF EXISTS Flights;')
cursor.execute('DROP TABLE IF EXISTS Bookings;')
cursor.execute('DROP TABLE IF EXISTS Payments;')
cursor.execute('DROP TABLE IF EXISTS MergedTable;')
# Create DataFrame for Customers
customer_data = pd.DataFrame({
    'Customer_ID': range(1, n + 1),
    'First_Name': [fake.first_name() for _ in range(n)],
    'Last_Name': [fake.last_name() for _ in range(n)],
    'Email': [fake.email() for _ in range(n)],
    'Phone': [fake.phone_number() for _ in range(n)]
})

# Create DataFrame for Flights
flight_data = pd.DataFrame({
    'Flight_ID': range(1, n + 1),
    'Departure_Location': [fake.city() for _ in range(n)],
    'Arrival_Location': [fake.city() for _ in range(n)],
    'Departure_Date': [fake.date_this_year() for _ in range(n)],
    'Price': np.random.randint(100, 1000, n)
})

# Create DataFrame for Bookings
booking_data = pd.DataFrame({
    'Booking_ID': range(1, n + 1),
    'Customer_ID': np.random.randint(1, n + 1, n),
    'Flight_ID': np.random.randint(1, n + 1, n),
    'Booking_Date': [fake.date_this_year() for _ in range(n)]
})

# Create DataFrame for Payments
payment_data = pd.DataFrame({
    'Payment_ID': range(1, n + 1),
    'Booking_ID': np.random.randint(1, n + 1, n),
    'Amount': np.random.uniform(50, 500, n),
    'Payment_Date': [fake.date_this_year() for _ in range(n)]
})

# Create SQLite database
conn = sqlite3.connect('TravelBookingDB.db')
cursor = conn.cursor()

# Create Customers table
cursor.execute('''
    CREATE TABLE Customers (
        Customer_ID INTEGER PRIMARY KEY,
        First_Name TEXT,
        Last_Name TEXT,
        Email TEXT,
        Phone TEXT
    );
''')
customer_data.to_sql('Customers', conn, index=False, if_exists='replace')

# Create Flights table
cursor.execute('''
    CREATE TABLE Flights (
        Flight_ID INTEGER PRIMARY KEY,
        Departure_Location TEXT,
        Arrival_Location TEXT,
        Departure_Date TEXT,
        Price REAL
    );
''')
flight_data.to_sql('Flights', conn, index=False, if_exists='replace')

# Create Bookings table
cursor.execute('''
    CREATE TABLE Bookings (
        Booking_ID INTEGER PRIMARY KEY,
        Customer_ID INTEGER,
        Flight_ID INTEGER,
        Booking_Date TEXT,
        FOREIGN KEY (Customer_ID) REFERENCES Customers(Customer_ID),
        FOREIGN KEY (Flight_ID) REFERENCES Flights(Flight_ID)
    );
''')
booking_data.to_sql('Bookings', conn, index=False, if_exists='replace')

# Create Payments table
cursor.execute('''
    CREATE TABLE Payments (
        Payment_ID INTEGER PRIMARY KEY,
        Booking_ID INTEGER,
        Amount REAL,
        Payment_Date TEXT,
        FOREIGN KEY (Booking_ID) REFERENCES Bookings(Booking_ID)
    );
''')
payment_data.to_sql('Payments', conn, index=False, if_exists='replace')

# Commit changes and close connection
conn.commit()
conn.close()
